### The Lifecycle Configuration Script downloads the jar file to the jdbc directory under home directory (~)

In [ ]:
jar_dir <- "~/SageMaker/jdbc"
jar_file <- list.files(jar_dir, pattern = "mariadb")
jar_dir_file <- file.path(jar_dir,jar_file)
libjvm_path <- system("find /usr/lib/jvm/ -name libjvm.so", intern=TRUE)
libjvm_path
dyn.load(libjvm_path)

options( java.parameters = "-Xmx8g" )

### Replace the ar_ep_port Address with the value of *AuroraClusterEndpointWithPort* in the CloudFormation Outputs

* For example:-
 * *ar_ep_port <- "blog-sagemaker-r-auroradb-cluster.cluster-XXXXXXXXXXXX.us-west-2.rds.amazonaws.com:3306"*

In [ ]:
ar_ep_port <- "<value of AuroraClusterEndpointWithPort in the CloudFormation Outputs>"

### Replace the ar_db with the value of *AuroraClusterDBName* in the Cloudformation Outputs.
#### Replace ar_user and ar_password with the *Secret Value* of *Secret Key* username and password in the AWS Secrets Manager Secret as denoted by *AuroraClusterSecret* in the CloudFormation Outputs

* For example:-
 * *ar_db <- "auroradb"*
 * *ar_user <- "auroramasteruser"*
 * *ar_password <- "XXXX"*

In [ ]:
ar_db <- "<value of AuroraClusterDBName in the Cloudformation Outputs>"
ar_user <- "<Secret Value of Secret Key 'username' of the AWS Secrets Manager Secret as denoted by AuroraClusterSecret in the CloudFormation Outputs>"
ar_password <- "<Secret Value of Secret Key 'password' of the AWS Secrets Manager Secret as denoted by AuroraClusterSecret in the CloudFormation Outputs>"


In [ ]:
jdbc_string <- paste("jdbc:mariadb:aurora//",ar_ep_port,"/",ar_db,sep="")
jdbc_string

In [ ]:
library(DBI)
library(rJava)
library(RJDBC)
options( java.parameters = "-Xmx8g" )
drv <- JDBC("org.mariadb.jdbc.Driver",jar_dir_file,identifier.quote="`")
conn <- dbConnect(drv, jdbc_string,ar_user,ar_password)

In [ ]:
show_databases <- dbGetQuery(conn, "show databases;")
show_databases

### Create a test table *students* in the the auroradb.

In [ ]:
res <- dbSendQuery(conn, "CREATE TABLE IF NOT EXISTS auroradb.students (name VARCHAR(64), age INT, gpa DECIMAL(3, 2));")
dbFetch(res)
dbClearResult(res)

### Insert couple of records in test table *students* in the the auroradb.

In [ ]:
res <- dbSendQuery(conn, "INSERT INTO auroradb.students VALUES ('fred flintstone', 35, 1.28), ('barney rubble', 32, 2.32);")
dbFetch(res)
dbClearResult(res)

In [ ]:
fetch=dbGetQuery(conn,"select * from auroradb.students;")
fetch